In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import gspread


In [ ]:
url = 'https://www.ptt.cc/bbs/WomenTalk/index.html'
response = requests.get(url, cookies={'oer18': '1'})
response.text
if response.status_code == 200:
  soup = BeautifulSoup(response.text, 'html.parser')
soup.find_all('div', class_='class')
for article in soup.find_all('div', class_='title'):
  print(article.text.strip())

[閒聊] 不會被AI淘汰的蠢(?
[閒聊] 拿坡里 必勝客 達美樂 都幾？
[閒聊] 相親失敗，會跟女方要回飯錢嗎？
[難過] 我是垃圾我今天好像什麼都還沒做
[閒聊] 沒人覺得活在現代也不錯嗎？
[閒聊] 想讓自己胸部變大吃雌激素可行嗎?
[心情] 放棄長輩後的調適與感情茫然
[閒聊] 有人看過恐怖家庭醫學嗎？
[閒聊] 明天的故事都是我的 是我的
[討論] Threads到底怎麼唸？
Fw: [公告] 關於利用網路擄人、囚禁等議題請板主宣導
[公告] 板規修訂預告，6/22起生效
[公告] WomenTalk板規 (2023.6.22)
Fw: [公告] EarthQuake43 地震專板討論


In [ ]:
def fetch_page(url):
  response = requests.get(url, cookies={'over18': '1'})  # 繞過年齡限制
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    for article in soup.find_all('div', class_='title'):
      print(article.text.strip())


In [ ]:
url = 'https://www.ptt.cc/bbs/WomenTalk/M.1713981763.A.66C.html'  # PTT 八卦版的一頁
fetch_page(url)

In [ ]:
def fetch_pages(start_page, end_page):
  BASE_URL = 'https://www.ptt.cc'
  current_page = f'{BASE_URL}/bbs/WomenTalk/index{start_page}.html'

  for _ in range(start_page, end_page + 1):
    response = requests.get(current_page, cookies={'over18': '1'})
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      for article in soup.find_all('div', class_='title'):
        print(article.text.strip())

      # 尋找上一頁的連結，並確保它存在
      prev_link = soup.find('a', string='‹ 上頁')
      if prev_link and 'href' in prev_link.attrs:
        current_page = BASE_URL + prev_link['href']
      else:
        print("無法找到上一頁的連結，或已達起始頁面。")
        break  # 如果沒有上一頁，或找不到連結，結束迴圈
    else:
      print("無法訪問頁面，請檢查連結或網絡狀態。")
      break

In [ ]:
fetch_pages(1, 10)

恭喜開版
ya!!!!開板了
輕舞飛揚
開版
恭喜...
哈...
Re: 輕舞飛揚
[公告]特此致謝
恭喜
[公告]關於 WomenTalk 板/女孩們的秘密花園
[閒聊] 恭喜開板
恭喜唷！
恭喜恭喜唷~~~~
耶~~唷唷
Re: 輕舞飛揚
恭喜
嗯嗯~~^__________^
[心得]我覺得身材真的不等於體重
恭喜開板
賀一下
無法找到上一頁的連結，或已達起始頁面。


In [ ]:
class PTTGossipCrawler:
  def __init__(self):
    self.BASE_URL = 'https://www.ptt.cc'
    self.session = requests.Session()
    self.session.cookies.update({'over18': '1'})  # 自動處理年齡限制
    self.data = []  # 用來儲存文章資訊

  def fetch_article_content(self, url):
    response = self.session.get(url)
    if response.status_code != 200:
      print(f"無法訪問文章頁面 {url}")
      return None

    soup = BeautifulSoup(response.text, 'html.parser')
    try:
      # 抓取文章資訊
      meta_values = soup.find_all('span', class_='article-meta-value')
      author = meta_values[0].text
      title = meta_values[2].text
      date = meta_values[3].text
      content = soup.find(id='main-content').text
      # 這裡簡化內文提取，實際上可能需要更複雜的處理來去除非內文部分
      return {'title': title, 'author': author, 'date': date, 'content': content}
    except Exception as e:
      print(f"解析文章頁面時出錯：{url}, 錯誤：{e}")
      return None

  def fetch_pages(self, start_page, end_page):
    current_page = f'{self.BASE_URL}/bbs/WomenTalk/index{start_page}.html'

    for _ in range(start_page, end_page + 1):
      response = self.session.get(current_page)
      if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        article_links = soup.find_all('div', class_='title')
        for article in article_links:
          a_tag = article.find('a')
          if a_tag:
            article_url = self.BASE_URL + a_tag['href']
            article_data = self.fetch_article_content(article_url)
            if article_data:
              self.data.append(article_data)

        prev_link = soup.find('a', string='‹ 上頁')
        if prev_link and 'href' in prev_link.attrs:
          current_page = self.BASE_URL + prev_link['href']
        else:
            break
      else:
          break

  def to_dataframe(self):
    return pd.DataFrame(self.data)

In [ ]:
crawler = PTTGossipCrawler()
crawler.fetch_pages(1, 10)  # 假定的頁面範圍
df = crawler.to_dataframe()

解析文章頁面時出錯：https://www.ptt.cc/bbs/WomenTalk/M.1051028274.A.AF9.html, 錯誤：list index out of range


In [ ]:
df['content'][0]

'作者goth (湖人贏啦)看板womentalk標題恭喜開版時間Wed Apr 23 00:12:00 2003\n\n--\n\n有什麼關係\n\n反正是夏天嘛\n\n--\n※ 發信站: 批踢踢實業坊(ptt.csie.ntu.edu.tw) \n◆ From: 218.187.32.242\n→ lladro::)                                           推  140.119.27.11 04/23\n推 idoit:..  59.115.207.59 09/27 20:26\n推 idoit:  59.115.207.59 09/27 20:35\n推 idoit:...  59.115.219.78 07/27 14:41\n→ unfearing:ID_MULTI版272和285   203.67.54.93 10/19 03:58\n推 chjna:000000000000 203.70.157.196 01/25 21:23\n推 Yimilin:... 04/29 12:16\n推 sokuso123:140.113.141.243 07/04 15:28\n噓 srxcomtx: 114.46.105.218 09/15 00:09\n推 johnny811025:  114.38.107.88 09/15 11:25\n推 mash1992: 114.38.113.202 09/16 14:52\n推 Bigfatty73  :首推   173.21.39.73 03/25 23:52\n推 NoPTT:\\⊙▽⊙/ 04/12 01:13\n推 lojyehuang: 12/14 20:18\n→ lojyehuang:如果你說的善良是指好人的話， 12/14 20:19\n→ lojyehuang:那我想，世上並沒有所謂真正的好人， 12/14 20:19\n→ lojyehuang:因為人們向來把對自己有用的人稱作好人， 12/14 20:19\n→ lojyehuang:但沒有人有辦法對所有人都有用， 12/14 20:19\n→ lojyehuang:所以這世上沒有真正善良的人的~~~ 12/14 20:19\n→ yourhighness:是否誤會了我的意思？？我從來都沒有說因為100元

In [ ]:
from google.colab import auth
auth.authenticate_user()

import google.auth
from google.auth.transport.requests import Request
from google.colab import auth
from oauth2client.client import GoogleCredentials
from gspread import authorize

creds, _ = google.auth.default()
gc = authorize(creds)

In [ ]:
def df_to_sheets(df, spreadsheet_id, sheet_name):
  try:
    sh = gc.open_by_key(spreadsheet_id)
  except Exception as e:
    print(f"Error opening the Spreadsheet: {e}")
    return

  try:
    worksheet = sh.worksheet(sheet_name)
  except gspread.WorksheetNotFound:
    worksheet = sh.add_worksheet(title=sheet_name, rows="100", cols="20")

  worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
spreadsheet_id = '13IXX3bap7S0ugjhNiWm7f77KzRNdP8ilLlPRUZB3v7A'
sheet_name = 'hw3_test'

def truncate_to_limit(cell_value, limit=10000):
  if len(cell_value) > limit:
    return cell_value[:limit] + '... [內容過長，已停止寫入]'
  else:
    return cell_value

In [ ]:
df_truncated = df.applymap(lambda x: truncate_to_limit(str(x)))
df_truncated

,title,author,date,content
0,恭喜開版,goth (湖人贏啦),Wed Apr 23 00:12:00 2003,作者goth (湖人贏啦)看板womentalk標題恭喜開版時間Wed Apr 23 00:...
1,ya!!!!開板了,winni318 (令人不齒的男人 丟臉),Wed Apr 23 00:16:42 2003,作者winni318 (令人不齒的男人 丟臉)看板womentalk標題ya!!!!開板了時...
2,開版,Earl (這只是ＩＤ),Wed Apr 23 00:22:51 2003,作者Earl (這只是ＩＤ)看板womentalk標題開版時間Wed Apr 23 00:2...
3,恭喜...,chrisdove (我要眠...眠....),Wed Apr 23 02:01:29 2003,作者chrisdove (我要眠...眠....)看板womentalk標題恭喜...時間W...
4,哈...,balanman (來砌磋五子棋吧~~),Wed Apr 23 02:30:19 2003,作者balanman (來砌磋五子棋吧~~)看板womentalk標題哈...時間Wed A...
5,Re: 輕舞飛揚,yinting (><),Wed Apr 23 07:48:53 2003,作者yinting (><)看板womentalk標題Re: 輕舞飛揚時間Wed Apr 2...
6,[公告]特此致謝,lladro (三點一刻),Wed Apr 23 08:32:37 2003,作者lladro (三點一刻)看板womentalk標題[公告]特此致謝時間Wed Apr ...
7,恭喜,mayge (狂牛部隊真強悍!!),Wed Apr 23 08:27:54 2003,作者mayge (狂牛部隊真強悍!!)看板womentalk標題恭喜時間Wed Apr 23...
8,[公告]關於 WomenTalk 板/女孩們的秘密花園,lladro (三點一刻),Wed Apr 23 09:07:02 2003,作者lladro (三點一刻)看板womentalk標題[公告]關於 WomenTalk 板...
9,[閒聊] 恭喜開板,ImOnLine (now),Wed Apr 23 12:17:40 2003,作者ImOnLine (now)看板womentalk標題[閒聊] 恭喜開板時間Wed Ap...


In [ ]:
df_to_sheets(df_truncated, spreadsheet_id, sheet_name)